In [ ]:
conda install -c conda-forge cudatoolkit==11.7.0
conda install -c conda-forge cudnn
conda install -c conda-forge cudatoolkit-dev
conda install -c conda-forge pytorch==1.13.1
conda install -c conda-forge nvidia-apex
git clone https://github.com/hpcaitech/ColossalAI.git
cd ColossalAI/applications/Chat
CUDA_EXT=1 pip install .
git clone https://github.com/Chenrj233/ParlAI.git
cd ParlAI
python setup.py install

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './checkpoint/llama_7B' \
    --model 'llama' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint/step1/epoch3' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 3 \
    --lora_rank 16 \
    --batch_size 2 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain 'facebook/opt-350m' \
    --model 'opt' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint_opt/step1/epoch3_test' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 3 \
    --batch_size 16 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch3' \
    --dataset 'Dahoas/rm-static' \
    --save_path './checkpoint_opt/step2/epoch3/rmstatic.pt' \
    --max_epochs 3 \
    --batch_size 16 \
    --loss_fn 'log_exp' \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch3' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_opt/step2/epoch3/rmpersona.pt' \
    --max_epochs 3 \
    --batch_size 16 \
    --loss_fn 'log_exp' \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=1 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaChat' \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch1' \
    --rm_model 'opt' \
    --rm_pretrain 'facebook/opt-2.7b' \
    --rm_path './checkpoint_opt/step2/epoch1/rmstatic.pt' \
    --save_path './checkpoint_opt/step3/epoch1/acstatic.pt' \
    --num_episodes 10 \
    --max_epochs 1 \
    --train_batch_size 8 \
    --ptx_batch_size 1 \
    --experience_batch_size 8\
    --lora_rank 16 \
    --max_input_len 511 \
    --max_seq_len 512 \
    --tokenizer 'facebook/opt-350m' \
    --max_datasets_size 16

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint 'persona_original' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint './checkpoint_opt/step1/epoch3' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10

In [ ]:
torchrun --standalone --nproc_per_node=1 inference.py \
    --model 'opt' \
    --pretrain 'facebook/opt-2.7b' \
    --model_path './checkpoint_opt/step3/epoch1/acstatic.pt' \
    --max_length 128 \
    --tokenizer 'facebook/opt-350m'   

In [ ]:
torchrun --standalone --nproc_per_node=1 inference.py \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch1' \
    --max_length 512 \
    --tokenizer 'facebook/opt-350m'   

In [ ]:
from transformers import AutoTokenizer
from transformers.models.opt.modeling_opt import OPTForCausalLM
self.tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m", add_prefix_space=True)
self.model_checkpoint = OPTForCausalLM.from_pretrained('../Chat/checkpoint_opt/step1/epoch1')

In [ ]:
epoch5
1. 0.03676635443136688
2. 0.02773737145198687

In [ ]:
docker run -it --gpus all -p 8899:8888 -e JUPYTER_ENABLE_LAB=yes -v /mnt/userhome/sun/jupyter:/home/jovyan/work persona_sun_v2 &